## TODO: Tutorial name
This is a tutorial showcasing the functionality of cooper in the context of a convex constrained optimization problem.

It is inspired by Engraved Blogpost

### Setup
Install cooper.

In [ ]:
!pip install git+https://github.com/gallego-posada/cooper
# TODO: .[examples], try from scratch
# !python setup.py install torch_cooper

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import torch
import cooper
from copy import deepcopy as copy

### Constrained Minimization Problem

Cooper supports constrained minimization problems with the following form:
\begin{align}
    & \underset{x}{\text{min}}\, & f(x) \tag{1} \\
    & \text{s.t.} & g(x) \leq 0 \\
    & & h(x) = 0
\end{align}

In this tutorial, we consider the following constrained optimization problem on
 the 2D domain $(x, y) \in [0,\pi/2] \times [0,\infty]$

\begin{align*}
\underset{x, y}{\text{min}}\quad f(x,y) &:= \left(1 - \text{sin}(x) \right) \ \big(1+y^2\big) & \tag{2} \\

s.t. \quad  g(x,y) &:= \left(1 - \text{cos}(x) \right)\ \big(1+y^2\big) \leq \epsilon & \\

\end{align*}

given some $\epsilon \geq 0 $.
Note how both $f$ and $g$ are convex functions in the specified domain.
As such, this constrained minimization problem is a convex problem.

In [ ]:
# Define a Constrained Minimization Problem. This object holds a state,
# containing the current values of the loss (f), inequality constraint defects (g),
# and equality constraint defects (h).


class Convex2dCMP(cooper.ConstrainedMinimizationProblem):
    def __init__(self, is_constrained=False, epsilon=1.0):
        self.epsilon = epsilon
        super().__init__(is_constrained)

    def closure(self, params):
        """This function evaluates the objective function and constraint
        defect. It updates the attributes of this CMP based on the results."""

        x = params[0]
        y = params[1]

        assert x >= 0 and x <= np.pi / 2
        assert y >= 0

        f = (1 - torch.sin(x)) * (1 + y**2)
        g = (1 - torch.cos(x)) * (1 + y**2)

        # Define the constraint defects
        _ineq_defect = g - self.epsilon  # in standard form (defect <= 0)
        _eq_defect = None  # no equality constraints

        # Store the values in a CMPState as attributes
        state = cooper.CMPState(
            loss=f,
            eq_defect=_eq_defect,
            ineq_defect=_ineq_defect,
            # other information can be stored in the misc dictionary. cooper
            # will not access misc and does not require it.
            misc={'g': g.data},
            )

        return state

In [ ]:
# Initialize the ConvexCMP.
EPSILON = 0.7
cmp = Convex2dCMP(is_constrained=True, epsilon=EPSILON)

# cmp.state is currently None, as we have not populated it yet.
print(cmp.state)

There are many ways to solve problems formulated as Eq. (1). In particular,
given our choice of problem in Eq. (2), we have convergence guarantees if we
employ this or that to solve it. can

Let us check the solution



### Formulation

Cooper has been designed to be used on neural network training in Pytorch.
As such, we use the Lagrangian formulation of Eq. (2) and optimize its respective
min-max game:

\begin{align*}
& \underset{x, y}{\text{min}}\ \underset{\lambda \geq 0}{\text{max}}\ f(x,y) +
\lambda \big( g(x,y) - \epsilon \big) \tag{3} \\
& \underset{x, y}{\text{min}}\ \underset{\lambda \geq 0}{\text{max}}\,
\big(1 - \text{sin}(x) \big) \ \big(1+y^2\big) +
\lambda \bigg( \big(1 - \text{cos}(x) \big)\ \big(1+y^2\big) - \epsilon \bigg) \\

\end{align*}

Now we setup a Lagrangian formulation for ConstrainedMinimizationProblem `cmp`

In [ ]:
# By default, multipliers are initialized to zero. Otherwise, we could provide
# ineq_init or eq_init for
formulation = cooper.LagrangianFormulation(cmp, ineq_init=None, eq_init=None)

# Lambda is held inside the formulation object.
multipliers = formulation.dual_parameters
print(multipliers)

### Optimizer

Now, construct the optimizer
Lagrangian via gradient descent ascent schemes.
Compatible with any torch optimizer on the primal and dual


In [ ]:
params = torch.nn.Parameter(torch.tensor([1.0, 1.0]))

In [ ]:
# Wrapper around torch optimizers
primal_optimizer = cooper.optim.SGD([params], lr=1e-2, momentum=0.0)
# Dual variables are initialized internally by the formulation when the shape
# of defects is identified. We provide a partial initializariion of the dual optim
dual_optimizer = cooper.optim.partial(cooper.optim.SGD, lr=1e-4)

# Initialize the constrained optimizer, which handles internally updates of the
# primal and dual variables.
constrained_optimizer = cooper.ConstrainedOptimizer(
    formulation=formulation,
    primal_optimizer=primal_optimizer,
    dual_optimizer=dual_optimizer,
)

### Training loop

The training loop is very similar to a vanilla pytorch loop

In [ ]:
# Store CMPStates and parameter values throughout the optimization process
params_history = [copy(params.data)]
multiplier_history = [0.]
loss_history = []
g_history = []

iters = range(0, 1001)
for iter in iters:
    # Zero the gradients wrt the parameters and multipliers
    constrained_optimizer.zero_grad()

    # Calculate the lagrangian.
    lagrangian = formulation.composite_objective(
        cmp.closure, # forward-like function, computing loss and defects
        params=params, # args and kwargs required for closure
        )

    # cmp.state has been updated inplace by formulation.composite_objective
    loss_history.append(cmp.state.loss.item())
    g_history.append(cmp.state.misc['g'].item())

    # Backward
    formulation.custom_backward(lagrangian)

    # Step
    constrained_optimizer.step()

    # Clip parameters
    min = torch.tensor([0.0, 0.0])
    max = torch.tensor([np.pi / 2, np.inf])
    params.data.clamp_(min=min, max=max)

    # Store new parameter values
    params_history.append(copy(params.data))
    multiplier = formulation.dual_parameters[0]
    multiplier_history.append(copy(multiplier.item()))

    if iter % 200 == 0:
        # Print progress
        print("\n Iteration: {}".format(iter))
        print(f"  Loss: {cmp.state.loss.item(): 0.3e}")
        print(f"  Defect: {copy(cmp.state.ineq_defect.item()): 0.3e}")

# CMPState after the final iteration
final_state = cmp.closure(params)

loss_history.append(final_state.loss.item())
g_history.append(final_state.misc['g'].item())


In [216]:
np.stack(params_history)

array([[1.        , 1.        ],
       [1.0108061 , 0.9968294 ],
       [1.0213957 , 0.99378407],
       ...,
       [1.5012965 , 0.40976575],
       [1.501255  , 0.40918794],
       [1.5012136 , 0.40861064]], dtype=float32)

In [218]:
iters

range(0, 1001)

## Training dynamics

TODO:
- grid of plots for these initial 3
- hline for optimal solution
- ipywidgets
- move bulk to plot_helpers.py


In [ ]:
iters_array = np.array([0, *iters])

In [ ]:
plt.plot(iters_array, loss_history)
plt.title("Loss")

In [ ]:
plt.plot(iters_array, g_history)
plt.axhline(EPSILON, color='r', linestyle='--')
plt.title(r"$g(x,y)$")

In [ ]:
plt.plot(iters_array, multiplier_history)
plt.title(r"$\lambda$")

# Fancier visualizations
- Parameters in param space with level curves
- f vs g